# Часть 2. Анализ тональности

In [1]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [2]:
!pip install pymorphy2 nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 961.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 28.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=3344c517a1e313a4335119cb7be20f90666cc4f19165f13e47ca30aefe36325b
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
import nltk
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from collections import Counter

Импортируем также стоп-слова, чтобы удалить их из отзывов. Так наш результат станет чище

In [5]:
russian_stopwords = stopwords.words("russian")

Предобрабатываем текст отзыва

In [58]:
# Инициализируем лемматизатор
morph = MorphAnalyzer()

def preprocess_text(data):
    # Токенизация
    tokens = word_tokenize(data)
    # Приведение к нижнему регистру
    tokens = [token.lower() for token in tokens]
    # Лемматизация
    lemmas = []
    for token in tokens:
      lemma = morph.normal_forms(token)[0]
      if lemma not in russian_stopwords: # Удаоение стоп-слов
        if lemma not in (',', '.', '!', '"', '?', ';', ':', '-', '–'):
          lemmas.append(lemma)

    return lemmas

# применяем функцию предобработки к столбцу 'text'
df_lemmatized = df.copy()
df_lemmatized['text'] = df_lemmatized['text'].apply(preprocess_text)

df_lemmatized

,Unnamed: 0,text,label
0,0,"[мочь, сказать, точно, книга, хотя, хороший, ф...",0
1,1,"[сказать, всё, смешаться, дом, облонской, ...,...",0
2,2,"[очень, поучительный, книга, взаимоотношение, ...",0
3,3,"[книга, очень, интересный, читаться, легко, за...",0
4,4,"[второй, часть, «, сумеречный, сага, », второй...",0
...,...,...,...
235,235,"[очень, интересный, книга, замечательный, ирла...",1
236,236,"[родитель, который, хотеть, обогатить, фантази...",1
237,237,"[увлекательный, книга, сочетать, жанр, фантаст...",1
238,238,"[произведение, заполниться, первый, очередь, ф...",1


Делим предобработанный датафрейм на тестовую и трейновую выборки.

In [59]:
from sklearn.model_selection import train_test_split
# у нас нет У_train / test, потому что в качестве айтема у нас целая строка датафрейма
X_train, X_test = train_test_split(df_lemmatized, test_size=0.2, random_state=42)

Для трейновой выборки соберем все слова, которые встречаются в позитивных и негативных отзывах

In [60]:
def unique_words(X_train):
  pos = [] # слова из позитивных отзывов

  for rew in X_train[X_train['label'] == 1]['text']:
      pos.extend(rew)

  neg = [] # cлова из негативных отзывов

  for rew in X_train[X_train['label'] == 0]['text']:
      neg.extend(rew)

  only_neg = set(neg) - set(pos)
  only_pos = set(pos) - set(neg)

  neg_freq = {}
  pos_freq = {}

  for text in X_train['text']:
    for word in text:
      if word in only_neg:
        if word not in neg_freq.keys():
          neg_freq[word]=1
        else:
          neg_freq[word]+=1
      elif word in only_pos:
        if word not in pos_freq.keys():
          pos_freq[word]=1
        else:
          pos_freq[word]+=1
      else:
        pass

  neg_count = [w[0] for w in Counter(neg_freq).most_common(100)] # выберем 100 самых частотных слов
  pos_count = [w[0] for w in Counter(pos_freq).most_common(100)]

  return neg_count, pos_count


In [61]:
neg_count = unique_words(X_train)[0]
pos_count = unique_words(X_train)[1]

Определим простую функцию классификации: будем считать, каких слов встретилось больше: из списка уникально-положительных или уникально-отрицательных.

In [62]:
def simple_prediction(test_df):
  global pos_count, neg_count

  X_pred = []
  p = 0
  n = 0

  for text in test_df['text']:

    for wor in text:
      if wor in pos_count:
        p+=1
      elif wor in neg_count:
        n+=1
      else:
        pass

    if p>n:
      X_pred.append(1)
    else:
      X_pred.append(0)

  return X_pred

Accuracy около 70%. Причина понятна -- в наши множества попало много мусорных слов, которые ничего не говорят о тональности текста.

In [63]:
X_pred = simple_prediction(X_test)

accuracy_score(list(X_test['label']), X_pred)

0.6875

Попробуем выделить из отзывов ключевые слова по методу RAKE и смотреть, принадлежат ли **они** к спискам.

In [64]:
!pip3 install rake-nltk
from rake_nltk import Rake

In [65]:
r = Rake()

In [66]:
def kw_prediction(test_df):
  global pos_count, neg_count


  X_pred = []

  p = 0
  n = 0

  for text in test_df['text']:

    # выделяем ключевые слова или фразы (в случае фраз делим их по пробелам)
    r.extract_keywords_from_text(' '.join(text))
    keywords = r.get_ranked_phrases_with_scores()

    kw = sum([key[1].split(' ') for key in keywords], [])

    for wor in kw:
      if wor in pos_count:
        p+=1
      elif wor in neg_count:
        n+=1
      else:
        pass

    if p>n:
      X_pred.append(1)
    else:
      X_pred.append(0)

  return X_pred

Accuracy упала :( Хотя и ожидалось, что станет лучше (потому что мы проводим проверку слов на "важность" как бы два раза: сначала ограничивая их частотность в словаре уникальных слов, а потом еще и выделением ключевых слов в самом отзыве). Видимо, выделение ключевых слов не помогает для текстов такого маленького размера -- а даже мешает, потому что может удалить информативные для тональности слова.  

In [67]:
X_pred_1 = kw_prediction(X_test)

accuracy_score(list(X_test['label']), X_pred_1)

0.6041666666666666

Еще одна идея по улучшению -- учитывать не только наличие / отсутствие слова в списке позитивных / негативных слов, но и частотность слова в этих списках.

Посчитаем уникальные негативные / позитивные слова снова, но не в виде множества, а учитывая их изначальную частотность в текстах

In [68]:
neg_count_freq = dict(Counter(neg_count))
for k, v in neg_count_freq.items():
  neg_count_freq[k]=neg_count_freq[k]/100

pos_count_freq = dict(Counter(pos_count))
for k, v in pos_count_freq.items():
  pos_count_freq[k]=pos_count_freq[k]/100


In [69]:
def freq_prediction(test_df):
  global pos_count_freq, neg_count_freq


  X_pred = []
  p = 0
  n = 0

  for text in test_df['text']:
    for wor in text:
      if wor in pos_count_freq.keys():
        p+=pos_count_freq[wor]
      elif wor in neg_count_freq.keys():
        n+=neg_count_freq[wor]
      else:
        pass


    if p>n:
      X_pred.append(1)
    else:
      X_pred.append(0)

  return X_pred

Accuracy не отличается от первого способа.

In [70]:
X_pred_2 = freq_prediction(X_test)

accuracy_score(list(X_test['label']), X_pred_2)

0.6875

Давайте теперь попробуем в изначальном датафрейме добавлять в выборку не все слова, а только ключевые. Для этого переделаем тестовую и трейновую выборки и пересчитаем все метрики заново.

In [39]:
def kw_preprocess_text(data):
    # Токенизация
    tokens = word_tokenize(data)
    # Приведение к нижнему регистру
    tokens = [token.lower() for token in tokens]
    # Лемматизация
    lemmas = []
    for token in tokens:
      lemma = morph.normal_forms(token)[0]
      if lemma not in russian_stopwords:
        if lemma not in (',', '.', '!', '"', '?', ';', ':', '-', '–'):
          lemmas.append(lemma)

    r.extract_keywords_from_text(' '.join(lemmas))
    keywords = r.get_ranked_phrases_with_scores()

    kw = list(set(sum([key[1].split(' ') for key in keywords[:3]], [])))

    return kw

# Применяем функцию предобработки к столбцу 'text'
df_kw = df.copy()
df_kw['text'] = df_kw['text'].apply(kw_preprocess_text)

# Вывод результата (для проверки)
df_kw

,Unnamed: 0,text,label
0,0,"[однако, сходить, вообще, хороший, точно, науч...",0
1,1,"[сделать, ещё, ...., ..., враг, смешаться, тре...",0
2,2,"[взаимоотношение, учебник, поступок, поучитель...",0
3,3,"[...., пора, отшельник, чудо, большой, понять,...",0
4,4,"[девушка, героиня, сумеречный, второй, взаимоо...",0
...,...,...,...
235,235,"[глаз, увы, целое, сесилия, замечательный, сле...",1
236,236,"[краска, стаж, родитель, 100, янушко, волшебны...",1
237,237,"[главное, сочетать, подросток, описывать, крас...",1
238,238,"[безысходность, фраза, любимый, жизнь, болезне...",1


In [40]:
X_train_1, X_test_1 = train_test_split(df_kw, test_size=0.2, random_state=42)

In [41]:
neg_count = unique_words(X_train_1)[0]
pos_count = unique_words(X_train_1)[1]

In [42]:
X_pred_21 = simple_prediction(X_test_1)

accuracy_score(list(X_test_1['label']), X_pred_21)

0.6458333333333334

In [43]:
X_pred_22 = kw_prediction(X_test_1)

accuracy_score(list(X_test_1['label']), X_pred_22)

0.6458333333333334

Accuracy функций чуть-чуть упала по сравнению с простым каунтом (и чуть возросла по сравнению с каунтом по ключевым словам в отзыве) Видимо, выделять ключевые слова лучше и в изначальных текстах, и в отзывах (но этот способ всё равно не самый лучший). Среди трех функций, берущих в виде инпута датафрейм с ключевыми словами, accuracy не меняется.

Важно, что качество в целом всё равно падает: ключевые слова опять обрезают нам часть информативных слов, как это было выше.

In [44]:
neg_count_freq = dict(Counter(neg_count))
for k, v in neg_count_freq.items():
  neg_count_freq[k]=neg_count_freq[k]/100

pos_count_freq = dict(Counter(pos_count))
for k, v in pos_count_freq.items():
  pos_count_freq[k]=pos_count_freq[k]/100

In [45]:
X_pred_23 = freq_prediction(X_test_1)

accuracy_score(list(X_test_1['label']), X_pred_23)

0.6458333333333334

Интересно, падает ли качество, если не удалять стоп-слова?

In [46]:
def with_sw(data):
    # Токенизация
    tokens = word_tokenize(data)
    # Приведение к нижнему регистру
    tokens = [token.lower() for token in tokens]
    # Лемматизация
    lemmas = []
    for token in tokens:
      lemma = morph.normal_forms(token)[0]
      lemmas.append(lemma)

    return lemmas

# Применяем функцию предобработки к столбцу 'text'
df_sw = df.copy()
df_sw['text'] = df_sw['text'].apply(with_sw)

# Вывод результата (для проверки)
df_sw

,Unnamed: 0,text,label
0,0,"[мочь, сказать, точно, ,, что, книга, хотя, бы...",0
1,1,"[можно, сказать, -, всё, смешаться, в, дом, об...",0
2,2,"[очень, поучительный, книга, о, взаимоотношени...",0
3,3,"[книга, очень, интересный, !, читаться, легко,...",0
4,4,"[второй, часть, «, сумеречный, сага, », ,, вто...",0
...,...,...,...
235,235,"[очень, интересный, книга, замечательный, ирла...",1
236,236,"[для, родитель, ,, который, хотеть, обогатить,...",1
237,237,"[увлекательный, книга, ., сочетать, в, себя, м...",1
238,238,"[я, произведение, заполниться, в, первый, очер...",1


In [47]:
X_train_2, X_test_2 = train_test_split(df_sw, test_size=0.2, random_state=42)

In [48]:
neg_count = unique_words(X_train_2)[0]
pos_count = unique_words(X_train_2)[1]

In [49]:
neg_count_freq = dict(Counter(neg_count))
for k, v in neg_count_freq.items():
  neg_count_freq[k]=neg_count_freq[k]/100

pos_count_freq = dict(Counter(pos_count))
for k, v in pos_count_freq.items():
  pos_count_freq[k]=pos_count_freq[k]/100

Удивительно, но точность падает не слишком сильно. Но сильно она падает для функции, выделяющей ключевые слова -- видимо, она выделяет в качестве ключевых как раз в том числе стоп-слова и знаки препинания

In [50]:
X_pred_31 = simple_prediction(X_test_2)

accuracy_score(list(X_test_2['label']), X_pred_31)

0.6458333333333334

In [51]:
X_pred_32 = kw_prediction(X_test_1)

accuracy_score(list(X_test_2['label']), X_pred_32)

0.5833333333333334

In [52]:
X_pred_33 = freq_prediction(X_test_2)

accuracy_score(list(X_test_2['label']), X_pred_33)

0.6458333333333334

Итак, для **действительно хорошей** класиификации тональности нужно:
- учитывать частотность слов
- скорее всего, учитывать их "важность" в исходных текстах, но иным от нашего способом
- удалять стоп-слова (более качественно, чем это делает нлтк)

Это всё приводит к идее о TF-IDF-векторизации отзывов и последующей их классификации при помощи метода kNN.